E7 Exercise 

In [8]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

file_path = r"ESILVtransportation_parameters.csv"
df = pd.read_csv(file_path)

Northwest corner rule

In [9]:
def northwest_corner_rule(df):
    try:
        # Extract unique sources and destinations
        sources = sorted(set(df["Source"]))
        destinations = sorted(set(df["Destination"]))
        
        # Create a cost matrix
        num_sources = len(sources)
        num_destinations = len(destinations)
        cost_matrix = np.zeros((num_sources, num_destinations))
        
        for index, row in df.iterrows():
            source_index = sources.index(row["Source"])
            dest_index = destinations.index(row["Destination"])
            cost_matrix[source_index][dest_index] = row["Cost"]
        
        # Initialize supply and demand arrays
        supply = np.array(df.groupby("Source")["Supply"].sum())
        demand = np.array(df.groupby("Destination")["Demand"].sum())
        
        # Initialize allocation matrix
        allocations = np.zeros_like(cost_matrix)
        
        # Apply Northwest Corner Rule
        source_index = 0
        dest_index = 0
        
        while source_index < num_sources and dest_index < num_destinations:
            quantity = min(supply[source_index], demand[dest_index])
            allocations[source_index][dest_index] = quantity
            supply[source_index] -= quantity
            demand[dest_index] -= quantity
            
            if supply[source_index] == 0:
                source_index += 1
            if demand[dest_index] == 0:
                dest_index += 1
        
        return allocations
    
    except Exception as e:
        print("An error occurred:", e)
        return None
    
# Northwest Corner Rule method
nw_allocation = northwest_corner_rule(df)
print("Northwest Corner Rule Allocation:\n", nw_allocation)

Northwest Corner Rule Allocation:
 [[20.  0.  0.]
 [10. 20. 25.]
 [ 0.  0.  0.]]


Minimum cost method 

In [10]:
def minimum_cost_method(df):
    try:
        # Extract unique sources, destinations, supply, demand, and costs
        sources = sorted(set(df["Source"]))
        destinations = sorted(set(df["Destination"]))
        supply = np.array(df.groupby("Source")["Supply"].sum())
        demand = np.array(df.groupby("Destination")["Demand"].sum())
        
        # Create a cost matrix
        num_sources = len(sources)
        num_destinations = len(destinations)
        costs = np.zeros((num_sources, num_destinations))
        
        for index, row in df.iterrows():
            source_index = sources.index(row["Source"])
            dest_index = destinations.index(row["Destination"])
            costs[source_index][dest_index] = row["Cost"]
        
        # Initialize allocation matrix
        allocations = np.zeros_like(costs)
        
        # Iterate until all supply and demand are satisfied
        while np.any(supply > 0) and np.any(demand > 0):
            min_cost = np.inf
            min_source, min_dest = -1, -1
            
            for i in range(num_sources):
                for j in range(num_destinations):
                    if supply[i] > 0 and demand[j] > 0 and costs[i][j] < min_cost:
                        min_cost = costs[i][j]
                        min_source, min_dest = i, j
                        
            quantity = min(supply[min_source], demand[min_dest])
            allocations[min_source][min_dest] = quantity
            supply[min_source] -= quantity
            demand[min_dest] -= quantity
        
        return allocations
    
    except Exception as e:
        print("An error occurred:", e)
        return None
        
min_cost_allocation = minimum_cost_method(df)
print("Minimum Cost Method Allocation:\n", min_cost_allocation)

Minimum Cost Method Allocation:
 [[ 0.  0. 20.]
 [ 0. 20.  5.]
 [30.  0.  0.]]


Minimum row cost method

In [11]:
def minimum_row_cost_method(df):
    try:
        # Extract unique sources, destinations, supply, demand, and costs
        sources = sorted(set(df["Source"]))
        destinations = sorted(set(df["Destination"]))
        supply = np.array(df.groupby("Source")["Supply"].sum())
        demand = np.array(df.groupby("Destination")["Demand"].sum())
        
        # Create a cost matrix
        num_sources = len(sources)
        num_destinations = len(destinations)
        costs = np.zeros((num_sources, num_destinations))
        
        for index, row in df.iterrows():
            source_index = sources.index(row["Source"])
            dest_index = destinations.index(row["Destination"])
            costs[source_index][dest_index] = row["Cost"]
        
        # Initialize allocation matrix
        allocations = np.zeros_like(costs)
        
        # Iterate over rows
        for i, row in enumerate(sources):
            # Find the column index with the minimum cost
            min_col_index = np.argmin(costs[i])
            
            # Allocate units based on minimum of supply and demand
            quantity = min(supply[i], demand[min_col_index])
            allocations[i][min_col_index] = quantity
            supply[i] -= quantity
            demand[min_col_index] -= quantity
        
        return allocations
    
    except Exception as e:
        print("An error occurred:", e)
        return None

    # Minimum Row Cost Method
min_row_allocation = minimum_row_cost_method(df)
print("Minimum Row Cost Method Allocation:\n", min_row_allocation)

Minimum Row Cost Method Allocation:
 [[20.  0.  0.]
 [ 0. 20.  0.]
 [10.  0.  0.]]


Vogel's method 

In [12]:
def vogels_method(df):
    try:
        # Extract unique sources, destinations, supply, demand, and costs
        sources = sorted(set(df["Source"]))
        destinations = sorted(set(df["Destination"]))
        supply = np.array(df.groupby("Source")["Supply"].sum())
        demand = np.array(df.groupby("Destination")["Demand"].sum())
        
        # Create a cost matrix
        num_sources = len(sources)
        num_destinations = len(destinations)
        costs = np.zeros((num_sources, num_destinations))
        
        for index, row in df.iterrows():
            source_index = sources.index(row["Source"])
            dest_index = destinations.index(row["Destination"])
            costs[source_index][dest_index] = row["Cost"]
        
        # Initialize allocation matrix
        allocations = np.zeros_like(costs)
        
        # Implement Vogel's Approximation Method (VAM)
        for _ in range(num_sources + num_destinations - 1):
            # Calculate the penalty for each row and column
            row_penalties = np.array([np.inf] * num_sources)
            col_penalties = np.array([np.inf] * num_destinations)
            
            for i in range(num_sources):
                row = costs[i]
                row_sorted = np.sort(row)
                if row_sorted[0] < np.inf:
                    row_penalties[i] = row_sorted[1] - row_sorted[0]
            
            for j in range(num_destinations):
                col = costs[:, j]
                col_sorted = np.sort(col)
                if col_sorted[0] < np.inf:
                    col_penalties[j] = col_sorted[1] - col_sorted[0]
            
            max_row_penalty = np.argmax(row_penalties)
            max_col_penalty = np.argmax(col_penalties)
            
            if row_penalties[max_row_penalty] >= col_penalties[max_col_penalty]:
                # Allocate to the cell with the minimum cost in the column
                min_cost_index = np.argmin(costs[:, max_col_penalty])
                quantity = min(supply[min_cost_index], demand[max_col_penalty])
                allocations[min_cost_index][max_col_penalty] = quantity
                supply[min_cost_index] -= quantity
                demand[max_col_penalty] -= quantity
                costs[min_cost_index][max_col_penalty] = np.inf
            else:
                # Allocate to the cell with the minimum cost in the row
                min_cost_index = np.argmin(costs[max_row_penalty])
                quantity = min(supply[max_row_penalty], demand[min_cost_index])
                allocations[max_row_penalty][min_cost_index] = quantity
                supply[max_row_penalty] -= quantity
                demand[min_cost_index] -= quantity
                costs[max_row_penalty][min_cost_index] = np.inf
        
        return allocations
    
    except Exception as e:
        print("An error occurred:", e)
        return None

vogel_allocation = vogels_method(df)
print("Vogel's Method Allocation:\n", vogel_allocation)

Vogel's Method Allocation:
 [[ 0.  0. 20.]
 [ 0. 20.  0.]
 [30.  0.  0.]]
